In [1]:
from time import time
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
import torchvision
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary
from copy import deepcopy


In [2]:
def train_test(model, trainload, testload, loss, optim, epochs, device):
    start_time = time()
    print(f'Start Time: {start_time}')

    for epoch in range(epochs):
        model.train()
        total_step = len(trainload)
        running_loss, running_corrects = 0, 0

        for i, (inputs, labels) in enumerate(trainload):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # train
            optim.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs) # prediction
                _loss = loss(outputs, labels) # loss
                _, pred = torch.max(outputs, 1)
                
                # backpropr
                _loss.backward()
                optim.step()

            running_loss += _loss.item() * inputs.size(0)
            running_corrects += torch.sum(pred == labels.data)

            # display every 100 samples
            if (i * inputs.size(0)) % 1000 == 0 and i != 0:
                print(f'Epoch {epoch+1}/{epochs}, Step {(i * inputs.size(0))}/{len(trainload.dataset)}, '
                      f'Loss: {running_loss/(i * inputs.size(0)):.2f}, Acc: {running_corrects.double()/(i * inputs.size(0)):.2f}')



        # valdation
        model.eval()
        with torch.no_grad():
            valid_loss, valid_acc = 0.0, 0.0

            for i, (inputs, labels) in enumerate(testload):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # test
                with torch.set_grad_enabled(False):
                    outputs = model(inputs)
                    _, pred = torch.max(outputs, 1)
                    _loss = loss(outputs, labels)

                valid_loss += _loss.item() * inputs.size(0)
                valid_acc += torch.sum(pred == labels.data)

            print(f'Epoch #{epoch+1}, '
                    f'Validation Loss: {valid_loss/len(testload.dataset):.2f}, '
                    f'Validation Acc: {valid_acc.double()/len(testload.dataset):.2f}')
            
    print(f'Training time: {start_time - time()}')
    print('Savin model')
    torch.save(model.state_dict(), 'my_shufflenet.pt')


In [3]:
def main():
    model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=True)
    model.eval()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # freeze all model params
    for params in model.parameters():
        params.requires_grad = False

    numftrs = model.fc.in_features
    model.fc = nn.Linear(numftrs, 10)

    model = model.to(device)

    train_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465),
            (0.247, 0.243, 0.261)
        )
    ])

    test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465),
            (0.247, 0.243, 0.261)
        )
    ])

    batch_size = 32

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=train_transforms)
    trainloader = DataLoader(trainset, batch_size=batch_size,shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=test_transforms)
    testloader = DataLoader(testset, batch_size=batch_size,shuffle=False)

    train_test(
        model=model,
        trainload=trainloader,
        testload=testloader,
        loss=nn.CrossEntropyLoss(),
        optim=optim.Adam(model.parameters(), lr=1e-3),
        epochs=35,
        device=device
    )
    print('==END==')


In [4]:
main()

Using cache found in /home/local/ASUAD/nkondepu/.cache/torch/hub/pytorch_vision_v0.10.0
/home/local/ASUAD/nkondepu/anaconda3/envs/cen598/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/local/ASUAD/nkondepu/anaconda3/envs/cen598/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified
Start Time: 1698296860.6688967
Epoch 1/35, Step 4000/50000, Loss: 2.24, Acc: 0.36
Epoch 1/35, Step 8000/50000, Loss: 2.15, Acc: 0.49
Epoch 1/35, Step 12000/50000, Loss: 2.07, Acc: 0.55
Epoch 1/35, Step 16000/50000, Loss: 2.01, Acc: 0.58
Epoch 1/35, Step 20000/50000, Loss: 1.95, Acc: 0.60
Epoch 1/35, Step 24000/50000, Loss: 1.89, Acc: 0.61
Epoch 1/35, Step 28000/50000, Loss: 1.84, Acc: 0.63
Epoch 1/35, Step 32000/50000, Loss: 1.79, Acc: 0.64
Epoch 1/35, Step 36000/50000, Loss: 1.75, Acc: 0.64
Epoch 1/35, Step 40000/50000, Loss: 1.71, Acc: 0.65
Epoch 1/35, Step 44000/50000, Loss: 1.67, Acc: 0.66
Epoch 1/35, Step 48000/50000, Loss: 1.63, Acc: 0.66
Epoch #1, Validation Loss: 1.18, Validation Acc: 0.73
Epoch 2/35, Step 4000/50000, Loss: 1.21, Acc: 0.72
Epoch 2/35, Step 8000/50000, Loss: 1.18, Acc: 0.72
Epoch 2/35, Step 12000/50000, Loss: 1.16, Acc: 0.72
Epoch 2/35, Step 16000/50000, Loss: 1.15, Acc: 0.72
Epoch 2/35,